# Explore feature space

## Setup

In [ ]:
# import libraries
import os
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pqdm.processes import pqdm
from tqdm.notebook import tqdm

import utils

In [ ]:
# load wind farm
nt = len(utils.wt9_x)
power_installed = (
    utils.wfm_low.windTurbines.powerCtFunction.power_ct_tab[0].max() / 1e9 * nt
)

In [ ]:
# visualise wind distribution
_ = utils.wfm_low.site.plot_wd_distribution(n_wd=8, ws_bins=[0, 5, 10, 15, 20, 25, 30])

## Baseline value

In [ ]:
# run simulation
sim_res = utils.run_sim()
print(
    f"LCoE [USD/MWh]: {(utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN) * power_installed / (sim_res.aep().sum().values * 1000):,.3f}"
)
print(
    f"Capacity factor [%]: {sim_res.aep().sum().values / (power_installed * 365.25 * 24)*100:,.2f}"
)
sim_res

## Assess across coarse range of yaw combinations for all turbines

In [ ]:
coarse_dir = os.path.join("data", "coarse_sweep.h5")
yaw_ind = {f"yaw_{i}": [-5, 0, 5] for i in sim_res.wt.values}
if os.path.isfile(coarse_dir):
    ds = xr.open_dataset(coarse_dir)
else:
    # initialise dataset
    ds = xr.Dataset(
        data_vars={
            "lcoe": ([f"yaw_{i}" for i in sim_res.wt.values], np.zeros((3,) * nt)),
            "cap_fac": ([f"yaw_{i}" for i in sim_res.wt.values], np.zeros((3,) * nt)),
            "aep": (
                ["wt"] + [f"yaw_{i}" for i in sim_res.wt.values],
                np.zeros((nt,) + (3,) * nt),
            ),
        },
        coords={"wd": list(sim_res.wt.values)} | yaw_ind,
    )
    # calculate across full range of options
    yaw_options = list(product(*yaw_ind.values()))
    simulations = {
        k: v
        for k, v in zip(
            yaw_options,
            pqdm(
                [{"yaw": x} for x in yaw_options],
                utils.run_sim,
                n_jobs=int(0.75 * os.cpu_count()),
                argument_type="kwargs",
            ),
        )
    }
    for yaw_option, sim_res in simulations.items():
        ds["lcoe"].loc[*yaw_option] = (
            (utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN)
            * power_installed
            / (sim_res.aep().sum().values * 1000)
        )
        ds["cap_fac"].loc[*yaw_option] = sim_res.aep().sum().values / (
            power_installed * 365.25 * 24
        )
        ds["aep"].loc[:, *yaw_option] = sim_res.aep().sum(["wd", "ws"]).values
    # save file
    ds.to_netcdf(coarse_dir)

In [ ]:
# flatten and sort values
flat_lcoe = ds.lcoe.stack(all_dims=yaw_ind.keys())
yaw_sorted = flat_lcoe["all_dims"][np.argsort(flat_lcoe.values)]
display(yaw_sorted)
yaw_opt = yaw_sorted[0].values.tolist()

In [ ]:
# plot distribution of lcoe values
_ = ds.lcoe.plot.hist(bins=25)

In [ ]:
# plot distribution of capacity factor values
_ = ds.cap_fac.plot.hist(bins=25)

In [ ]:
# run simulation using optimum yaw
sim_res_opt = utils.run_sim(yaw=yaw_opt)
print(
    f"LCoE [USD/MWh]: {(utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN) * power_installed / (sim_res_opt.aep().sum().values * 1000):,.3f}"
)
print(
    f"Capacity factor [%]: {sim_res_opt.aep().sum().values / (power_installed * 365.25 * 24)*100:,.2f}"
)

In [ ]:
# plot flow map
fig, ax = plt.subplots()
fm = utils.run_sim(yaw=yaw_opt, ws=12).flow_map()
fm.plot_wake_map(ax=ax)
ax.grid()
fig.tight_layout()
fm